<a href="https://colab.research.google.com/github/IrisMejuto/deep_learning/blob/main/Practica_DL_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# 0. Información de las versiones usadas en el notebook
import os
import pandas as pd
from google.colab import drive
import cv2
import folium
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.utils
from tensorboard import program
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Optuna version: {optuna.__version__}")

PyTorch version: 2.6.0+cu124
Torchvision version: 0.21.0+cu124
NumPy version: 2.0.2
Pandas version: 2.2.2
Optuna version: 4.3.0


In [ ]:
import os
import pandas as pd
from google.colab import drive

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir la ruta base a la carpeta de tu proyecto
BASE_PATH = '/content/drive/MyDrive/ProyectoDL'

# 3. Construir la ruta al archivo de datos usando os.path.join
data_path = os.path.join(BASE_PATH, 'data', 'poi_dataset.csv')

# 4. Cargar los datos
try:
    df = pd.read_csv(data_path)
    print("¡Archivo cargado exitosamente!")
    print(f"Ruta del archivo: {data_path}")
    print(df.head())
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en la ruta '{data_path}'")
    print("Por favor, verifica que la ruta y el nombre del archivo son correctos.")


Mounted at /content/drive
¡Archivo cargado exitosamente!
Ruta del archivo: /content/drive/MyDrive/ProyectoDL/data/poi_dataset.csv
                                     id  \
0  4b36a3ed-3b28-4bc7-b975-1d48b586db03   
1  e32b3603-a94f-49df-8b31-92445a86377c   
2  0123a69b-13ac-4b65-a5d5-71a95560cff5   
3  390d7d9e-e972-451c-b5e4-f494af15e788   
4  023fc1bf-a1cd-4b9f-af78-48792ab1a294   

                                                name  \
0                                  Galería Fran Reus   
1                            Convento de San Plácido   
2             Instituto Geológico y Minero de España   
3                              Margarita Gil Roësset   
4  Museo del Traje. Centro de Investigación del P...   

                                    shortDescription  \
0  La Galería Fran Reus es un espacio dedicado a ...   
1  El Convento de San Plácido en Madrid, fundado ...   
2  El Instituto Geológico y Minero de España, sit...   
3  Margarita Gil Roësset, escultora y poetisa esp.

## Preparación del dataset


In [ ]:
# Carga de datos
data_path = os.path.join(BASE_PATH, 'data', 'poi_dataset.csv')
df = pd.read_csv(data_path)

In [ ]:
# Muestra dataset
df.head()

,id,name,shortDescription,categories,tier,locationLon,locationLat,tags,xps,Visits,Likes,Dislikes,Bookmarks,main_image_path
0,4b36a3ed-3b28-4bc7-b975-1d48b586db03,Galería Fran Reus,La Galería Fran Reus es un espacio dedicado a ...,"['Escultura', 'Pintura']",1,2.642262,39.572694,[],500,10009,422,3582,78,data_main/4b36a3ed-3b28-4bc7-b975-1d48b586db03...
1,e32b3603-a94f-49df-8b31-92445a86377c,Convento de San Plácido,"El Convento de San Plácido en Madrid, fundado ...","['Patrimonio', 'Historia']",1,-3.704467,40.423037,[],500,10010,7743,96,2786,data_main/e32b3603-a94f-49df-8b31-92445a86377c...
2,0123a69b-13ac-4b65-a5d5-71a95560cff5,Instituto Geológico y Minero de España,"El Instituto Geológico y Minero de España, sit...","['Ciencia', 'Patrimonio']",2,-3.699694,40.442045,[],250,10015,3154,874,595,data_main/0123a69b-13ac-4b65-a5d5-71a95560cff5...
3,390d7d9e-e972-451c-b5e4-f494af15e788,Margarita Gil Roësset,"Margarita Gil Roësset, escultora y poetisa esp...",['Cultura'],1,-3.691228,40.427256,[],500,10011,8559,79,2358,data_main/390d7d9e-e972-451c-b5e4-f494af15e788...
4,023fc1bf-a1cd-4b9f-af78-48792ab1a294,Museo del Traje. Centro de Investigación del P...,"El Museo del Traje de Madrid, fundado en 2004,...","['Patrimonio', 'Cultura']",1,-3.727822,40.439665,[],500,10020,915,2896,143,data_main/023fc1bf-a1cd-4b9f-af78-48792ab1a294...


In [ ]:
# Información general del dataset
print("\nInformación del dataset:")
df.info()


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1569 entries, 0 to 1568
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1569 non-null   object 
 1   name              1569 non-null   object 
 2   shortDescription  1569 non-null   object 
 3   categories        1569 non-null   object 
 4   tier              1569 non-null   int64  
 5   locationLon       1569 non-null   float64
 6   locationLat       1569 non-null   float64
 7   tags              1569 non-null   object 
 8   xps               1569 non-null   int64  
 9   Visits            1569 non-null   int64  
 10  Likes             1569 non-null   int64  
 11  Dislikes          1569 non-null   int64  
 12  Bookmarks         1569 non-null   int64  
 13  main_image_path   1569 non-null   object 
dtypes: float64(2), int64(6), object(6)
memory usage: 171.7+ KB


In [ ]:
# Verificamos valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())


Valores nulos por columna:
id                  0
name                0
shortDescription    0
categories          0
tier                0
locationLon         0
locationLat         0
tags                0
xps                 0
Visits              0
Likes               0
Dislikes            0
Bookmarks           0
main_image_path     0
dtype: int64


In [ ]:
# Dimensiones (filas, columnas)
print("\nDimensión Dataset:")
df.shape


Dimensión Dataset:


(1569, 14)

In [ ]:
# Tipos de datos únicos en columnas categóricas
print("\nValores únicos en columnas relevantes:")
print("Tiers únicos:", df['tier'].unique())
print("Tags únicos:", df['tags'].unique())
print("Número de categorías únicas:", len(set(category for categories_list in df['categories'].apply(eval) for category in categories_list)))
print("Número de tags únicos:", len(set(tag for tags_list in df['tags'].apply(eval) for tag in tags_list)))



Valores únicos en columnas relevantes:
Tiers únicos: [1 2 3 4]
Tags únicos: ['[]' "['s.XX']"
 "['Canaletto', 'paisaje', 'vedutismo', 's.XVIII', 'Cuadro', 'Venecia']"
 ... "['Estatuas', 'Plaza']" "['Alcalá', 'Gran Vía', 'Metrópolis']"
 "['Arquitectura', 'Cisterciense', 'Císter', 'Moreruela', 'Zamora', 'Ruinas', 'Monasterio']"]
Número de categorías únicas: 12
Número de tags únicos: 2935


In [ ]:
# Frecuencias de cada tag
all_tags = [tag for tags_list in df['tags'].apply(eval) for tag in tags_list]
tag_frequencies = pd.Series(all_tags).value_counts()

print("\nTags más frecuentes:")
print(tag_frequencies.head(10))


Tags más frecuentes:
Historia        802
Madrid          700
Cultura         583
Arquitectura    398
Patrimonio      353
Arte            329
Escultura       328
Estatua         269
Monumento       166
Naturaleza      150
Name: count, dtype: int64
